In [1]:
# 1. Mengimpor pustaka yang diperlukan
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# 2. Memuat dan Memproses Data
# Memuat dataset dari file CSV
try:
    df = pd.read_csv('Automobile(1).csv')
except FileNotFoundError:
    print("Pastikan file 'Automobile(1).csv' berada di direktori yang sama dengan script ini.")
    # Jika file tidak ditemukan, hentikan eksekusi atau berikan path yang benar
    # Contoh: df = pd.read_csv('path/to/your/Automobile(1).csv')
    exit()


# Mengganti nilai '?' yang sering digunakan untuk data hilang dengan NaN (Not a Number)
df.replace('?', np.nan, inplace=True)

# Mengidentifikasi kolom numerik dan kategorikal
numeric_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

# Mengisi nilai yang hilang pada kolom numerik dengan nilai rata-rata (mean)
for col in numeric_cols:
    if df[col].isnull().any():
        # Memastikan semua data di kolom adalah numerik, mengubah yang tidak valid menjadi NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].mean(), inplace=True)

# Mengisi nilai yang hilang pada kolom kategorikal dengan nilai yang paling sering muncul (mode)
for col in categorical_cols:
    if df[col].isnull().any():
        df[col].fillna(df[col].mode()[0], inplace=True)

# Mengubah kolom kategorikal menjadi numerik menggunakan Label Encoding
# Model machine learning memerlukan input berupa angka
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 3. Menyiapkan Data untuk Model
# Menentukan fitur (X) dan variabel target (y)
# 'price' adalah target yang ingin diprediksi
X = df.drop('price', axis=1)
y = df['price']

# Membagi data menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Membuat dan Melatih Model Machine Learning

# --- Model 1: Boosting (dengan GradientBoostingRegressor) ---
print("Melatih model Boosting...")
# Inisialisasi model
boosting_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
# Melatih model dengan data latih
boosting_model.fit(X_train, y_train)
# Membuat prediksi pada data uji
boosting_predictions = boosting_model.predict(X_test)


# --- Model 2: Bagging (dengan BaggingRegressor) ---
print("Melatih model Bagging...")
# Inisialisasi model
bagging_model = BaggingRegressor(n_estimators=100, random_state=42)
# Melatih model dengan data latih
bagging_model.fit(X_train, y_train)
# Membuat prediksi pada data uji
bagging_predictions = bagging_model.predict(X_test)


# 5. Mengevaluasi Performa Model

print("\n--- Hasil Evaluasi Model ---")

# Fungsi untuk menghitung dan mencetak metrik evaluasi
def evaluate_model(y_true, y_pred, model_name):
    """Fungsi untuk menghitung MSE, RMSE, dan R2."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    print(f"\n--- Evaluasi {model_name} ---")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R-squared (R2): {r2:.2f}")

# Mengevaluasi kedua model
evaluate_model(y_test, boosting_predictions, "Boosting (Gradient Boosting)")
evaluate_model(y_test, bagging_predictions, "Bagging")

<ipython-input-1-4052949261>:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


Melatih model Boosting...
Melatih model Bagging...

--- Hasil Evaluasi Model ---

--- Evaluasi Boosting (Gradient Boosting) ---
Mean Squared Error (MSE): 6075205.38
Root Mean Squared Error (RMSE): 2464.79
R-squared (R2): 0.92

--- Evaluasi Bagging ---
Mean Squared Error (MSE): 5354082.86
Root Mean Squared Error (RMSE): 2313.89
R-squared (R2): 0.93
